In [1]:
%matplotlib inline

In [2]:
import pandas as pd
from pandas_datareader import data as web #conda install -c anaconda pandas-datareader
import datetime
import os

In [149]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' #https://stackoverflow.com/questions/66092421/how-to-rebuild-tensorflow-with-the-compiler-flags

current_path = os.getcwd ()+'\\data\\'

START_YEAR = 2010
END_YEAR = 2022
START_DATE = "2010-01-01"
END_DATE = "2021-12-31"
WINDOW_SIZE = 30

# Some important moments from data collection 

_Please carefully try this because downloading a large amount of data!_

## 1. Get data for stationary features

In [112]:
tickers = pd.read_csv(os.path.join (current_path, "indicators_list.csv"), sep = ";")
print(f"All tickers is {len (tickers)}")
tickers = tickers.iloc[0:5,:] #Test only for first five tickers
tickers

All tickers is 98


,Ticker,Name,Type,Source,Frequency
0,^GSPC,S&P 500,Indices,Yahoo,Daily
1,^DJI,Dow 30,Indices,Yahoo,Daily
2,^IXIC,Nasdaq,Indices,Yahoo,Daily
3,^NYA,NYSE COMPOSITE (DJ),Indices,Yahoo,Daily
4,^VIX,CBOE Volatility Index,Indices,Yahoo,Daily


In [113]:
def parser(x):
    return datetime.datetime.strptime(x,'%Y-%m-%d')

In [114]:
def read_data_from_yahoo(ticker):
    start = datetime.datetime(START_YEAR,1,1)
    end = datetime.datetime(END_YEAR,1,1)
    print ('reading data of', ticker ,' from Yahoo Finance..')
    raw_data = web.DataReader(ticker, "yahoo", start, end)

    return raw_data

In [115]:
def add_data_to_df(ticker, stocks_df):
    df = pd.read_csv ( f'stocks_data\\{ticker}.csv', header=0, parse_dates=[0],
                             date_parser=parser )
    mask = (df['Date'] >= START_DATE) & (df['Date'] <= END_DATE)
    df = df.loc[mask]
    df = df[["Date", "Adj Close"]]
    df = df.rename({'Adj Close': ticker}, axis=1)
    df.set_index ( 'Date' )
    if len(stocks_df) == 0:
        stocks_df = df[["Date"]]
        stocks_df.set_index ( 'Date' )
    stocks_df = pd.merge ( stocks_df, df, how = 'left', left_on = 'Date', right_on = 'Date' )
    stocks_df = pd.merge ( stocks_df, df, how = 'left' )
    return stock_df

In [116]:
def scraping_data(stocks_df, tickers):
    for i in range ( 0, len ( tickers ) ):
        try:
            if tickers.Source[i] == "Yahoo":
                company_data = read_data_from_yahoo ( tickers.Ticker[i] )
                company_data.to_csv ( f'stocks_data\\{tickers.Ticker[i]}.csv', index="Data", header=True )
                stocks_df = add_data_to_df(tickers.Ticker[i], stock_df)
        except:
            continue
    return stocks_df

In [81]:
stocks_df = pd.DataFrame()
stocks_df = scraping_data(stock_df,tickers)
stocks_df.to_csv ( f'stocks_data\\dataset_full.csv', index="Data", header=True )
stocks_df

reading data of ^GSPC  from Yahoo Finance..
reading data of ^DJI  from Yahoo Finance..
reading data of ^IXIC  from Yahoo Finance..
reading data of ^NYA  from Yahoo Finance..
reading data of ^VIX  from Yahoo Finance..


,Date,^GSPC,^DJI,^IXIC,^NYA,^VIX
0,2010-01-04,1132.989990,10583.959961,2308.419922,7326.740234,20.040001
1,2010-01-05,1136.520020,10572.019531,2308.709961,7354.870117,19.350000
2,2010-01-06,1137.140015,10573.679688,2301.090088,7377.700195,19.160000
3,2010-01-07,1141.689941,10606.860352,2300.050049,7393.930176,19.059999
4,2010-01-08,1144.979980,10618.190430,2317.169922,7425.350098,18.129999
...,...,...,...,...,...,...
3016,2021-12-27,4791.189941,36302.378906,15871.259766,17141.089844,17.680000
3017,2021-12-28,4786.350098,36398.210938,15781.719727,17134.160156,17.540001
3018,2021-12-29,4793.060059,36488.628906,15766.219727,17149.929688,16.950001
3019,2021-12-30,4778.729980,36398.078125,15741.559570,17164.240234,17.330000


In [82]:
stocks_df.describe().T

,count,mean,std,min,25%,50%,75%,max
^GSPC,3021.0,2260.488112,890.501675,1022.580017,1461.400024,2088.479980,2798.360107,4793.060059
^DJI,3021.0,19756.317518,6927.100147,9686.480469,13557.000000,17851.509766,25332.179688,36488.628906
^IXIC,3021.0,6004.283709,3438.840186,2091.790039,3131.489990,4984.620117,7669.169922,16057.440430
^NYA,3021.0,10877.208197,2533.227013,6434.810059,8636.910156,10797.540039,12610.769531,17310.509766
^VIX,3021.0,18.130060,7.217589,9.140000,13.410000,16.299999,20.629999,82.690002


In [83]:
stocks_df.dtypes

Date     datetime64[ns]
^GSPC           float64
^DJI            float64
^IXIC           float64
^NYA            float64
^VIX            float64
dtype: object

The full dataset was processed by PyCharm. There had some hand made data manipulation and cleanig. They were selected 85 features, because lack of data. At the next step add inflation data. 

In [144]:
inf_data = pd.read_csv(r"data\inflation_data.csv", sep = ";")
inflation_df

,Date,T10YIE,T5YIE
0,2003-01-02,1.64,1.30
1,2003-01-03,1.62,1.28
2,2003-01-06,1.63,1.31
3,2003-01-07,1.62,1.28
4,2003-01-08,1.71,1.33
...,...,...,...
4967,2022-01-17,2.44,2.79
4968,2022-01-18,2.46,2.80
4969,2022-01-19,2.40,2.73
4970,2022-01-20,2.33,2.74


In [146]:
stocks_df = pd.read_csv(r"data\stocks_df.csv", sep = ";")
inf_data = inf_data.drop(labels=['Date'], axis=1)
stocks_df = stocks_df.merge(inf_data, left_index=True,right_index=True)

In [147]:
stocks_df[stocks_df.isna().any(axis=1)]

,Date,^GSPC,^DJI,^IXIC,^NYA,^VIX,000001.SS,EURUSD=X,JPY=X,GBPUSD=X,...,MCD,T,NKE,DHR,LOW,LIN,TXN,NEE,T10YIE,T5YIE


In [148]:
stocks_df.to_csv ( f'data\\stocks_df.csv', index = False, header=True )

## 2. Get data for current stock

In [9]:
# https://www.alpharithms.com/calculate-macd-python-272222/
def read_company_data_from_yahoo(ticker):
    start = datetime.datetime(START_YEAR,1,1)
    end = datetime.datetime(END_YEAR,1,1)
    print ('reading data of', ticker ,' from Yahoo Finance..')
    raw_data = web.DataReader(ticker, "yahoo", start, end)

    # Add Exponential moving average EWM
    ewm = pd.DataFrame ( raw_data['Adj Close'].ewm ( span=21, adjust=False ).mean () )
    raw_data['ewm'] = ewm

    # Add Moving average MA
    ma_7 = pd.DataFrame ( raw_data['Adj Close'].rolling(7).mean() )
    raw_data['ma_7'] = ma_7

    ma_21 = pd.DataFrame ( raw_data['Adj Close'].rolling(21).mean() )
    raw_data['ma_21'] = ma_21

    # calculate Relative Strength Index (RSI) momentum oscillator
    delta = raw_data['Adj Close'].diff()
    up = delta.clip(lower=0)
    down = -1 * delta.clip(upper=0)
    ema_up = up.ewm(com=13, adjust=False).mean()
    ema_down = down.ewm(com=13, adjust=False).mean()
    rs = ema_up / ema_down
    raw_data['rsi'] = 100 - (100 / (1 + rs))

    # calculate MACD
    # Get the 12-day EMA of the closing price
    k = raw_data['Adj Close'].ewm ( span=12, adjust=False, min_periods=12 ).mean ()
    # Get the 26-day EMA of the closing price  
    d = raw_data['Adj Close'].ewm ( span=26, adjust=False, min_periods=26 ).mean ()
    # Subtract the 26-day EMA from the 12-Day EMA to get the MACD
    macd = k - d

    raw_data['macd'] = macd

    # calculate Bollinger bands
    tp = (raw_data['Close'] + raw_data['Low'] + raw_data['High']) / 3
    std = tp.rolling ( 20 ).std ( ddof=0 )
    ma_tp = tp.rolling ( 20 ).mean ()
    bolu = ma_tp + 2 * std
    bold = ma_tp - 2 * std
    raw_data['bolu'] = bolu
    raw_data['bold'] = bold

    raw_data.to_csv ( f'stocks_data\\{ticker}_indicators.csv', index="Data", header=True )

    return raw_data

In [10]:
def preprocessing_data(companies):
    for i in range (0, len(companies)):
        company_data = read_company_data_from_yahoo(companies[i])
    return company_data

In [96]:
companies = ["ADM", "AAPL", "MSFT", "AMZN", "TSLA", "GOOGL"]
company_df =  preprocessing_data(companies)
print()
print(f"Example for GOOGL DataFrame")
company_df
# df =  read_company_data_from_yahoo("ADM")
# company_df.head()

reading data of ADM  from Yahoo Finance..
reading data of AAPL  from Yahoo Finance..
reading data of MSFT  from Yahoo Finance..
reading data of AMZN  from Yahoo Finance..
reading data of TSLA  from Yahoo Finance..
reading data of GOOGL  from Yahoo Finance..

Example for GOOGL DataFrame


,High,Low,Open,Close,Volume,Adj Close,ewm,ma_7,ma_21,rsi,macd,bolu,bold
Date,,,,,,,,,,,,,
2010-01-04,315.070068,312.432434,313.788788,313.688690,3908488.0,313.688690,313.688690,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-05,314.234222,311.081085,313.903900,312.307312,6003391.0,312.307312,313.563110,NaN,NaN,0.000000,NaN,NaN,NaN
2010-01-06,313.243256,303.483490,313.243256,304.434448,7949443.0,304.434448,312.733232,NaN,NaN,0.000000,NaN,NaN,NaN
2010-01-07,305.305298,296.621613,305.005005,297.347351,12815771.0,297.347351,311.334516,NaN,NaN,0.000000,NaN,NaN,NaN
2010-01-08,301.926941,294.849854,296.296295,301.311310,9439151.0,301.311310,310.423315,NaN,NaN,12.078872,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,2966.879883,2943.389893,2945.120117,2958.129883,798800.0,2958.129883,2905.393007,2892.821394,2893.850470,56.422249,4.398914,2985.316961,2803.013061
2021-12-28,2965.979980,2921.090088,2964.719971,2933.739990,910000.0,2933.739990,2907.970006,2899.227121,2898.139997,53.371218,6.114445,2989.204159,2802.671858
2021-12-29,2948.340088,2912.949951,2932.889893,2933.100098,889400.0,2933.100098,2910.254559,2913.312849,2899.210949,53.289799,7.337796,2992.574985,2806.114704


In [97]:
company_df.describe().T

,count,mean,std,min,25%,50%,75%,max
High,3021.0,8.819487e+02,6.313704e+02,221.361359,3.781782e+02,7.326800e+02,1.153240e+03,3.019330e+03
Low,3021.0,8.658072e+02,6.197569e+02,217.032028,3.705005e+02,7.166100e+02,1.131400e+03,2.977980e+03
Open,3021.0,8.740192e+02,6.255493e+02,219.374374,3.723474e+02,7.251500e+02,1.143700e+03,2.999510e+03
Close,3021.0,8.742112e+02,6.259783e+02,218.253250,3.727477e+02,7.248300e+02,1.145170e+03,2.996770e+03
Volume,3021.0,3.114318e+06,2.543914e+06,465600.000000,1.471800e+06,2.163100e+06,4.116080e+06,2.961995e+07
Adj Close,3021.0,8.742112e+02,6.259783e+02,218.253250,3.727477e+02,7.248300e+02,1.145170e+03,2.996770e+03
ewm,3021.0,8.656164e+02,6.146397e+02,233.830905,3.628427e+02,7.276951e+02,1.139923e+03,2.927754e+03
ma_7,3015.0,8.727362e+02,6.227538e+02,222.942943,3.730731e+02,7.253572e+02,1.143801e+03,2.966057e+03
ma_21,3001.0,8.693464e+02,6.153189e+02,233.572859,3.710568e+02,7.281510e+02,1.143053e+03,2.945134e+03
rsi,3020.0,5.415258e+01,1.258934e+01,0.000000,4.533362e+01,5.432212e+01,6.307415e+01,8.864022e+01


In [98]:
company_df.dtypes

High         float64
Low          float64
Open         float64
Close        float64
Volume       float64
Adj Close    float64
ewm          float64
ma_7         float64
ma_21        float64
rsi          float64
macd         float64
bolu         float64
bold         float64
dtype: object

## 3. Prepare time series

In [87]:
df = pd.read_csv(os.path.join (os.getcwd ()+"\\stocks_data\\", "ADM_indicators.csv"), sep = ",")
dataset = df.iloc[:,[0,6]]
dataset

,Date,Adj Close
0,2010-01-04,22.943090
1,2010-01-05,23.067034
2,2010-01-06,23.008707
3,2010-01-07,22.768120
4,2010-01-08,22.483797
...,...,...
3016,2021-12-27,66.070000
3017,2021-12-28,66.930000
3018,2021-12-29,67.010002
3019,2021-12-30,66.860001


In [83]:
def prepare_windows(df, window_size):
    sample = []
    ts_dataset = pd.DataFrame()
    for i in range(1, window_size+1):
        ts_dataset[f"Lag_{i}"] = df["Adj Close"].shift(i) 
    ts_dataset =  pd.concat([df, ts_dataset], axis=1)
    return ts_dataset.loc[window_size:, :]

In [104]:
dataset_ts = prepare_windows(dataset, WINDOW_SIZE)
company_df_full = dataset_ts.merge(df, left_index=True,right_index=True).drop(labels=["Date_y", "Adj Close_y"], axis=1)
company_df_full = company_df_full.rename(columns = {'Date_x': 'Date', 'Adj Close_x': 'Adj Close'}, inplace = False)
company_df_full = company_df_full.reset_index(drop=True, inplace=False)
company_df_full

,Date,Adj Close,Lag_1,Lag_2,Lag_3,Lag_4,Lag_5,Lag_6,Lag_7,Lag_8,...,Open,Close,Volume,ewm,ma_7,ma_21,rsi,macd,bolu,bold
0,2010-02-17,21.894228,21.601130,21.703720,21.893267,21.864101,21.805784,21.791199,22.316114,21.915140,...,29.620001,29.879999,4611900.0,22.055134,21.793347,22.021045,48.937402,-0.192514,31.180627,29.203706
1,2010-02-18,21.960175,21.894228,21.601130,21.703720,21.893267,21.864101,21.805784,21.791199,22.316114,...,29.719999,29.969999,3679600.0,22.046501,21.817486,22.006525,50.038546,-0.175643,31.079499,29.203501
2,2010-02-19,21.945518,21.960175,21.894228,21.601130,21.703720,21.893267,21.864101,21.805784,21.791199,...,29.860001,29.950001,3897500.0,22.037321,21.837448,21.978810,49.781613,-0.161593,31.023873,29.184127
3,2010-02-22,21.842934,21.945518,21.960175,21.894228,21.601130,21.703720,21.893267,21.864101,21.805784,...,29.830000,29.809999,3619800.0,22.019649,21.834424,21.962180,47.926619,-0.156927,30.981804,29.161863
4,2010-02-23,21.703722,21.842934,21.945518,21.960175,21.894228,21.601130,21.703720,21.893267,21.864101,...,29.809999,29.620001,5819300.0,21.990929,21.807347,21.937879,45.451485,-0.162588,30.980237,29.121097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2986,2021-12-27,66.070000,65.440002,64.620003,64.830002,64.220001,65.269997,66.169998,65.080002,64.849998,...,65.540001,66.070000,1175900.0,64.719121,65.231429,64.166191,58.105626,0.302337,66.388560,62.033441
2987,2021-12-28,66.930000,66.070000,65.440002,64.620003,64.830002,64.220001,65.269997,66.169998,65.080002,...,66.000000,66.930000,1049900.0,64.920110,65.340001,64.286667,61.772180,0.438992,66.711888,61.891113
2988,2021-12-29,67.010002,66.930000,66.070000,65.440002,64.620003,64.830002,64.220001,65.269997,66.169998,...,67.050003,67.010002,1038500.0,65.110100,65.588573,64.391429,62.104439,0.547438,67.116617,61.917051
2989,2021-12-30,66.860001,67.010002,66.930000,66.070000,65.440002,64.620003,64.830002,64.220001,65.269997,...,67.199997,66.860001,906900.0,65.269182,65.965716,64.612857,61.033303,0.614197,67.396706,62.084962


In [105]:
company_df_full[company_df_full.isna().any(axis=1)]

,Date,Adj Close,Lag_1,Lag_2,Lag_3,Lag_4,Lag_5,Lag_6,Lag_7,Lag_8,...,Open,Close,Volume,ewm,ma_7,ma_21,rsi,macd,bolu,bold


In [109]:
company_df_full.dtypes

Date          object
Adj Close    float64
Lag_1        float64
Lag_2        float64
Lag_3        float64
Lag_4        float64
Lag_5        float64
Lag_6        float64
Lag_7        float64
Lag_8        float64
Lag_9        float64
Lag_10       float64
Lag_11       float64
Lag_12       float64
Lag_13       float64
Lag_14       float64
Lag_15       float64
Lag_16       float64
Lag_17       float64
Lag_18       float64
Lag_19       float64
Lag_20       float64
Lag_21       float64
Lag_22       float64
Lag_23       float64
Lag_24       float64
Lag_25       float64
Lag_26       float64
Lag_27       float64
Lag_28       float64
Lag_29       float64
Lag_30       float64
High         float64
Low          float64
Open         float64
Close        float64
Volume       float64
ewm          float64
ma_7         float64
ma_21        float64
rsi          float64
macd         float64
bolu         float64
bold         float64
dtype: object

In [107]:
company_df_full.to_csv ( f'data\\company_dataset_ts.csv', index = False, header=True )